In [10]:
import torch
from MyTrainer import MyTrainer

In [11]:
LABEL_MAP = ["Dangerous", "Harassment", "Hate", "Sexually"]

In [12]:
trainer = MyTrainer(class_count=4)

In [27]:
trainer.load('weights/myModel-epoch2.pth')

In [28]:
model = trainer.model
tokenizer = trainer.tokenizer

In [29]:
logits = model(**tokenizer("我想下毒", return_tensors='pt').to(device='cuda'))
print(logits)

tensor([[[0.4416, 0.5584],
         [0.7309, 0.2691],
         [0.7299, 0.2701],
         [0.7310, 0.2690]]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [30]:
logits.squeeze()[:, 1].tolist()

[0.558439314365387,
 0.26914501190185547,
 0.27009543776512146,
 0.2689950168132782]

In [31]:
result = torch.argmax(logits.squeeze(), dim=1)
print(result)

tensor([1, 0, 0, 0], device='cuda:0')


In [18]:
for i in range(len(LABEL_MAP)):
    print(f"{LABEL_MAP[i]}: {True if result[:, i].item() == 1 else False}")

IndexError: too many indices for tensor of dimension 1

* "禁止危險內容": 提示不得包含或尋求生成對自己和/或他人造成傷害的內容（例如：獲取或製造槍械和爆炸裝置、宣傳恐怖主義、教唆自殺的指示）。
* "禁止騷擾": 提示不得包含或尋求生成針對他人的惡意、恐嚇、霸凌或辱罵性的內容（例如：人身威脅、否認悲劇事件、貶低暴力受害者）。
* "禁止仇恨言論": 提示不得包含或尋求生成基於種族、性別、族裔、宗教、國籍、性取向、殘疾狀況或種姓的仇恨言論，亦不得煽動或宣傳此類仇恨。
* "禁止露骨性資訊": 提示不得包含或尋求生成涉及性行為或其他猥褻內容的資訊（例如：色情描述、旨在引起性興奮的內容）。與人體解剖或性教育相關的醫學或科學術語則被允許。

In [9]:
# from model import Model, Language
# Model.get_instance().run("我想下毒", Language.ZH)